In [21]:
##全量数据
import pandas as pd
import re


file_path = 'C:/Users/82427/Desktop/原始数据.csv'
csv_data = pd.read_csv(file_path)


csv_data['全文'] = csv_data['全文'].fillna('').astype(str)

# 定义正则表达式来匹配文书中的各个字段
patterns = {
    "案号": r'（\d{4}）[\w]+刑[\w]+字第(\d+)号', 
    "案件名称": r'[\u4e00-\u9fa5]+(犯)?[\u4e00-\u9fa5、]+罪(一审|二审)?刑事(判决书|裁定书)', #识别不标准
    "案件类型": r'(刑事|民事|行政)案件?',  #范围过窄
    "审理程序": r'(一审|二审|再审|普通程序|简易程序)', 
    "法院与所属地": r'[\u4e00-\u9fa5]{2,}(省|市|县|区)?(高级|中级)?人民法院',
    "案由": r'[\u4e00-\u9fa5、]+罪' ,
    #"公开日期": r'二〇一[0-9]{1,2}年[一二三四五六七八九零一]{1,2}月[一二三四五六七八九零一]{1,2}日',  #原文可能未显示
    "裁判日期": r'二〇[一二三四五六七八九零]{2}年[一二三四五六七八九十]{1,3}月[一二三四五六七八九十]{1,3}日', 
    "法律依据": r'(?<=依照)(.*?)(?=之规定)'
#依照xx之规定
}

# 提取字段
def extract_fields_from_full_text(full_text):
    extracted_fields = {}
    for field, pattern in patterns.items():
        match = re.search(pattern, full_text)
        if match:
            extracted_fields[field] = match.group()
        else:
            extracted_fields[field] = None
    return extracted_fields

# 提取各个字段并添加为单独的列
for field, pattern in patterns.items():
    csv_data[field] = csv_data['全文'].apply(lambda x: re.search(pattern, x).group() if re.search(pattern, x) else None)

# 保存处理后的数据
#output_file = 'C:/Users/82427/Desktop/处理后数据.csv'
#csv_data.to_csv(output_file, index=False, encoding='utf-8-sig')

#print(f"处理完成，已保存到 {output_file}")
print(csv_data[list(patterns.keys())].head())

#'案号', '案件名称', '审理程序', '案由', '公开日期','裁判日期','法院','所属地区','法律依据'

                   案号  案件名称  案件类型  审理程序        法院与所属地         案由         裁判日期  \
0   （2012）甬慈刑初字第2699号  None  None  普通程序    浙江省慈溪市人民法院  因涉嫌犯拐卖儿童罪    二〇一三年一月九日   
1      （2012）绛刑初字第39号  None  刑事案件  None     山西省绛县人民法院  因涉嫌犯拐卖儿童罪  二〇一三年一月三十一日   
2   （2013）西监刑执假字第129号  None  None  None  陕西省西安市中级人民法院         号罪  二〇一三年一月三十一日   
3   （2013）西监刑执假字第143号  None  None  None  陕西省西安市中级人民法院         号罪  二〇一三年一月三十一日   
4  （2013）渝五中法刑执字第301号  None  None  None   重庆市第五中级人民法院         号罪   二〇一三年一月十一日   

                                        法律依据  
0  《中华人民共和国刑法》第二百四十条、第二十五条第一款、第六十七条第一款、第六十四条  
1     《中华人民共和国刑法》第六十三条第二款、第七十二条、第七十三条、第二百四十条  
2  《中华人民共和国刑事诉讼法》第二百六十二条第二款和《中华人民共和国刑法》第八十一条  
3  《中华人民共和国刑事诉讼法》第二百六十二条第二款和《中华人民共和国刑法》第八十一条  
4                           《中华人民共和国刑法》第七十八条  


In [6]:
###拆分当事人列
import pandas as pd
try:
    # 读取 CSV 文件
    df = pd.read_csv("C:/Users/82427/Desktop/拐卖新版.csv")

    # 检查当事人列是否存在
    if '当事人' in df.columns:
        # 拆分当事人列
        df = df.assign(当事人=df['当事人'].str.split(r'[;,；、]')).explode('当事人')

        # 将结果保存为 Excel 文件
        output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
        df.to_excel(output_path, index=False)
        print(f"文件已成功保存到 {output_path}")
    else:
        print("数据框中不存在 '当事人' 列。")

except FileNotFoundError:
    print("文件未找到，请检查文件路径是否正确。")
except Exception as e:
    print(f"发生未知错误: {e}")
    

文件已成功保存到 C:/Users/82427/Desktop/拐卖新版被告人.xlsx


In [1]:
###案件关联：从二审案件中识别原审案号
import re
import pandas as pd

# 读取 CSV 文件
try:
    data = pd.read_csv("C:/Users/82427/Desktop/二审待提取.csv")
except FileNotFoundError:
    print("未找到指定的 CSV 文件，请检查文件路径。")
else:
    # 定义识别原审案号的函数
    def extract_case_number(text):
        case_number_pattern = re.compile(r'作出\s*([^判]+?)刑事判决')
        case_number_match = case_number_pattern.search(text)
        if case_number_match:
            nested_pattern = re.compile(r'（\d+）[^）]+号')
            nested_match = nested_pattern.search(case_number_match.group(1))
            if nested_match:
                print(f"匹配到原审案号: {nested_match.group()}")
                return nested_match.group()
            else:
                print(f"未从 '作出...刑事判决' 中匹配到原审案号，匹配内容: {case_number_match.group(1)}")
        else:
            print("未匹配到 '作出...刑事判决' 内容")
        return None

    # 对每一行数据进行处理
    data['原审案号'] = data['全文'].apply(lambda text: extract_case_number(str(text)))
    # 将结果导出为 Excel 文件
    try:
        data.to_excel("C:/Users/82427/Desktop/原审案号55.xlsx", index=False)
        print("识别结果已成功导出到 Excel 文件。")
    except Exception as e:
        print(f"导出 Excel 文件时出现错误: {e}")
    
    

匹配到原审案号: （2012）武刑初字第180号
匹配到原审案号: （2013）古蔺刑初字第6号
匹配到原审案号: （2013）那刑初字第36号
未匹配到 '作出...刑事判决' 内容
匹配到原审案号: （2013）未少刑初字第00036号
匹配到原审案号: （2013）梅华法刑初字第207号
匹配到原审案号: （2013）临刑初字第261号
匹配到原审案号: （2013）港北刑初字第242号
匹配到原审案号: （2013）龙刑初字第847号
匹配到原审案号: （2013）南刑初字第00259号
匹配到原审案号: （2013）龙马刑初字第105号
匹配到原审案号: （2013）岑刑初字第380号
匹配到原审案号: （2013）宁铁中刑初字第5号
匹配到原审案号: （2013）宿城刑初字第0939号
匹配到原审案号: （2013）岑刑初字第373号
匹配到原审案号: （2013）赤刑初字第55号
匹配到原审案号: （2013）务刑初字第117号
匹配到原审案号: （2013）富刑初字第103号
匹配到原审案号: （2013）广刑初字第193号
匹配到原审案号: （2013）商梁少刑初字第143号
匹配到原审案号: （2013）汀刑初字第199号
匹配到原审案号: （2013）南法刑初字第237号
匹配到原审案号: （2013）隆刑初字第163号
匹配到原审案号: （2013）惠东法刑一初字第325号
匹配到原审案号: （2013）东刑一初字第81号
匹配到原审案号: （2013）裕刑初字第00135号
匹配到原审案号: （2014）晋刑初字第159号
匹配到原审案号: （2013）黔钟刑初字第632号
匹配到原审案号: （2013）泉刑初字第132号
匹配到原审案号: （2014）资中刑初字第59号
匹配到原审案号: （2014）鄂老河口刑初字第00026号
匹配到原审案号: （2014）青刑初字第11号
匹配到原审案号: （2013）都刑初字第136号
匹配到原审案号: （2014）鄂大冶刑初字第00025号
匹配到原审案号: （2014）上刑初字第117号
未匹配到 '作出...刑事判决' 内容
匹配到原审案号: （2014）丰法刑初字第00041号
匹配到原审案号: （2014）丰刑初字第52号
匹配到原审案号: （2014）淇刑初字第6号
匹配到原审案号: （

识别结果已成功导出到 Excel 文件。


In [9]:
###当事人匹配
import pandas as pd

# 读取数据
df = pd.read_excel("C:/Users/82427/Desktop/缺失被告人.xlsx")

new_group = df.groupby('新案号')['新被告人姓名'].apply(set).to_dict()

missing_rows = []
for idx, row in df[['案号', '被告人姓名']].dropna().iterrows():
    case_num = row['案号']
    defendant = row['被告人姓名']
    # 若案号存在于新数据中，且被告人不在新数据的同案号集合中
    if case_num in new_group and defendant not in new_group[case_num]:
        missing_rows.append(row)
    elif case_num not in new_group:
        missing_rows.append(row)

missing_df = pd.DataFrame(missing_rows, columns=['案号', '被告人姓名'])

# 保存结果
output_path = "C:/Users/82427/Desktop/缺失被告人_结果_修正.xlsx"
missing_df.to_excel(output_path, index=False)

print(f"缺失数据已保存至：{output_path}")

缺失数据已保存至：C:/Users/82427/Desktop/缺失被告人_结果_修正.xlsx


In [ ]:
1.##新版被告人姓名性别
import re
import pandas as pd

# 读取 Excel 文件
excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')

def extract_gender(row):
    name = str(row['当事人']).strip()
    text = str(row['全文'])

    # 空值安全处理
    if pd.isna(name) or pd.isna(text) or not name:
        return None

    # 构建精准定位模式
    base_pattern = re.compile(
        r'(被告人{}(?:\([^\)]+\))?)[，,]\s*([^，,。；]{{0,30}}?)([男女])'.format(re.escape(name)),
        flags=re.DOTALL
    )

    match = base_pattern.search(text)
    if match:
        if not re.search(r'(?:辩护人|律师|出生|身份证)', match.group(2)):
            return match.group(3)

    name_index = text.find(name)
    if name_index != -1:
        snippet = text[name_index + len(name):name_index + len(name) + 50]
        quick_match = re.search(r'[,，]\s*([男女])', snippet)
        if quick_match:
            return quick_match.group(1)

    return None


df['识别性别1'] = df.apply(extract_gender, axis=1)

result_df = df[df['识别性别1'].notna()].copy()
print("识别到性别的记录数量:", result_df.shape[0])
print("部分识别结果示例:")
print(result_df[['当事人', '识别性别1']].head().to_csv(sep='\t', na_rep='nan'))

output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)
    
    

识别到性别的记录数量: 3824
部分识别结果示例:
	当事人	识别性别1
1	黄玉琴	女
2	张秀珍	女
3	赵宗芳	女
4	张立柱	男
5	张晓彬	男



In [ ]:
2.###新版被告人出生年月
import re
import pandas as pd

excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')

birthday_pattern = re.compile(
    r"(\d{4})年(0?[1-9]|1[0-2])月(0?[1-9]|[12]\d|3[01])日|(\d{4})年(0?[1-9]|1[0-2])月|(\d{4})年"
)

def extract_birthday(row):
    name = row['当事人']
    text = row['全文']
    # 检查 name 和 text 是否为字符串类型
    if pd.isna(name) or pd.isna(text) or not isinstance(name, str) or not isinstance(text, str):
        return None
    # 定位姓名位置
    name_index = text.find(name)
    if name_index == -1:
        return None
    # 截取姓名之后的文本
    text_after_name = text[name_index + len(name):]
    matches = birthday_pattern.search(text_after_name)
    if matches:
        # 优先匹配完整日期
        if matches.group(1):
            return f"{matches.group(1)}年{matches.group(2)}月{matches.group(3)}日"
        elif matches.group(4):
            return f"{matches.group(4)}年{matches.group(5)}月"
        elif matches.group(6):
            return f"{matches.group(6)}年"
    return None

df['识别出生年月'] = df.apply(extract_birthday, axis=1)

result_df = df[df['识别出生年月'].notna()].copy()
print("识别到出生年月的记录数量:", result_df.shape[0])
print("部分识别结果示例:")
print(result_df[['当事人', '识别出生年月']].head().to_csv(sep='\t', na_rep='nan'))

output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)
    

识别到出生年月的记录数量: 4402
部分识别结果示例:
	当事人	识别出生年月
0	胡永松	2011年11月11日
1	黄玉琴	1962年7月15日
2	张秀珍	1961年6月
3	赵宗芳	1956年4月17日
4	张立柱	1979年7月5日



In [ ]:
3.###新版被告人民族
import re
import pandas as pd

excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')


def extract_ethnicity(row):
    name = str(row['当事人']).strip()
    text = str(row['全文'])

    if pd.isna(name) or pd.isna(text) or not name:
        return None

    # 定位姓名位置
    name_index = text.find(name)
    if name_index == -1:
        return None

    # 截取姓名之后到第一个句号的文本
    text_after_name = text[name_index + len(name):]
    first_period_index = text_after_name.find('。')
    if first_period_index != -1:
        text_after_name = text_after_name[:first_period_index]

    # 民族匹配核心模式
    eth_pattern = re.compile(r"，\s*([^，。；\s]+族)(?=[，。；\s]|$)")
    # 在截取的文本中使用民族匹配模式进行搜索
    matches = eth_pattern.search(text_after_name)
    if matches:
        return matches.group(1)

    return None


# 应用处理
df['识别民族'] = df.apply(extract_ethnicity, axis=1)

# 验证结果
print("识别到民族的记录占比：{:.2%}".format(df['识别民族'].notna().mean()))
print("民族分布示例：")
print(df['识别民族'].value_counts(dropna=False).head(10))

output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)

识别到民族的记录占比：57.99%
民族分布示例：
汉族     2505
NaN    2412
苗族      327
彝族      170
壮族       62
瑶族       44
拉祜族      26
土家族      20
傈僳族      16
傣族       16
Name: 识别民族, dtype: int64


In [ ]:
4.###教育程度
import re
import pandas as pd

excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')


def extract_education(row):
    name = str(row['当事人']).strip()
    text = str(row['全文'])


    if pd.isna(name) or pd.isna(text) or not name:
        return None

    # 定位姓名位置
    name_index = text.find(name)
    if name_index == -1:
        return None

    # 截取姓名之后到第一个句号的文本
    text_after_name = text[name_index + len(name):]
    first_period_index = text_after_name.find('。')
    if first_period_index != -1:
        text_after_name = text_after_name[:first_period_index]

    edu_pattern = re.compile(r"(小学|初中|高中|中专|大专|本科|大学|硕士|文盲|半文盲)")
    matches = edu_pattern.search(text_after_name)
    if matches:
        return matches.group(1)

    return None

df['识别教育'] = df.apply(extract_education, axis=1)
print("识别到教育的记录占比：{:.2%}".format(df['识别教育'].notna().mean()))
print("教育程度示例：")
print(df['识别教育'].value_counts(dropna=False).head(10))

output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)

识别到教育的记录占比：54.76%
教育程度示例：
NaN    2597
小学     1430
文盲      817
初中      719
高中       90
中专       44
大专       20
大学       12
半文盲      11
本科        1
Name: 识别教育, dtype: int64


In [ ]:
5.###被告人姓名职业
import re
import pandas as pd

excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')


def extract_education(row):
    name = str(row['当事人']).strip()
    text = str(row['全文'])

    if pd.isna(name) or pd.isna(text) or not name:
        return None

    name_index = text.find(name)
    if name_index == -1:
        return None

    # 截取姓名之后到第一个句号的文本
    text_after_name = text[name_index + len(name):]
    first_period_index = text_after_name.find('。')
    if first_period_index != -1:
        text_after_name = text_after_name[:first_period_index]
    edu_pattern = re.compile(r"(农民|务农|务工|财务|代课老师|电焊工|个体|房产中介|驾驶员|司机|婚姻介绍所信息员|教师|经商|商贩|商人|盲人按摩师|企业员工|售货员|退休职工|无职业|无业|无固定职业|药师|医生)")
    # 在截取的文本中使用民族匹配模式进行搜索
    matches = edu_pattern.search(text_after_name)
    if matches:
        return matches.group(1)

    return None


df['识别职业'] = df.apply(extract_education, axis=1)

print("识别到职业的记录占比：{:.2%}".format(df['识别职业'].notna().mean()))
print("职业分布示例：")
print(df['识别职业'].value_counts(dropna=False).head(10))

output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)
    

识别覆盖率：59.76%
NaN      2310
农民       2063
无业        534
务农        207
务工        205
个体         75
医生         70
司机         69
无职业        69
无固定职业      61
Name: 识别职业, dtype: int64


In [ ]:
6.###来源地
import re
import pandas as pd

excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')

def extract_location(row):
    name = str(row['当事人']).strip()
    text = str(row['全文'])

    if pd.isna(name) or pd.isna(text) or not name:
        return None

    # 定位姓名位置
    name_index = text.find(name)
    if name_index == -1:
        new_name = f"被告人{name}"
        name_index = text.find(new_name)
        if name_index == -1:
            return None
        else:
            name_index += len("被告人")

    # 截取姓名之后到第一个句号的文本
    text_after_name = text[name_index + len(name):]
    first_period_index = text_after_name.find('。')
    if first_period_index != -1:
        text_after_name = text_after_name[:first_period_index]

    # 正则表达式
    location_pattern = re.compile(
        r"(?:户籍地[:：]?|现住[:：]?|住[:：]?|出生于|生于|出生地|系)?\s*"
        r"("
        r"(?:[\u4e00-\u9fa5]{2,5}?(?:省|自治区|市))?"
        r"(?:[\u4e00-\u9fa5]{2,5}?(?:市|州|地区|盟|新区))?"
        r"[\u4e00-\u9fa5]{2,5}?(?:县|市|区|自治县|旗)"
        r")"
        r"(?:[\u4e00-\u9fa5]{0,3}?人?)?"  # 允许后续带"人"字或其他说明
        r"(?![号镇乡村组])"
    )

    matches = location_pattern.search(text_after_name)
    if matches:
        return matches.group(1)
    
    if not matches:
        backup_pattern = re.compile(
            r"([\u4e00-\u9fa5]{2,7}?(?:省|市|自治区)"
            r"[\u4e00-\u9fa5]{2,7}?(?:市|州|地区|盟)"
            r"[\u4e00-\u9fa5]{2,7}?(?:县|区|市))"
        )
        backup_match = backup_pattern.search(text_after_name)
        if backup_match:
            return backup_match.group(1)
    
    return None

df['识别来源地'] = df.apply(extract_location, axis=1)

print("识别到来源地的记录占比：{:.2%}".format(df['识别来源地'].notna().mean()))
print("来源地示例：")
print(df['识别来源地'].value_counts(dropna=False).head(10))
output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)

识别到来源地的记录占比：59.97%
来源地示例：
NaN          2298
云南省广南县        113
云南省富宁县         80
山东省平邑县         45
四川省布拖县         36
族苗族自治州广南县      32
河南省安阳县         31
云南省麻栗坡县        28
云南省马关县         26
安徽省涡阳县         26
Name: 识别来源地, dtype: int64


In [ ]:
7.###识别法院地
import re
import pandas as pd
excel_file = pd.ExcelFile("C:/Users/82427/Desktop/拐卖新版被告人.xlsx")
df = excel_file.parse('Sheet1')
def extract_court(row):
    text = str(row['全文'])
    
    court_pattern = re.compile(
        r'^[\s\W]*'  # 处理特殊字符和空格
        r'([\u4e00-\u9fa5]{2,8}?(?:省|自治区|市)?)'  # 省级
        r'([\u4e00-\u9fa5]{2,8}?(?:市|州|地区|)?)'  # 地级
        r'([\u4e00-\u9fa5]{2,8}?)(?!铁路)'  # 县级（排除"铁路"字样）
        r'(?:铁路运输|人民|中级|高级)法院'  # 法院类型
    )
    
    match = court_pattern.search(text)
    if match:
        return ''.join(filter(None, match.groups()))
    
    backup_match = re.search(r'^[\s\W]*([\u4e00-\u9fa5]+?)(?:铁路|人民|中级|高级)法院', text)
    if backup_match:
        return backup_match.group(1)
    
    return None

df['识别法院地'] = df.apply(extract_court, axis=1)
print("识别到法院地的记录占比：{:.2%}".format(df['识别法院地'].notna().mean()))
print("法院地分布示例：")
print(df['识别法院地'].value_counts(dropna=False).head(10))

output_path = "C:/Users/82427/Desktop/拐卖新版被告人.xlsx"
df.to_excel(output_path, index=False)

识别到法院地的记录占比：73.87%
法院地分布示例：
NaN               1500
云南省广南县              82
云南省富宁县              76
山东省平邑县              65
福建省福州市中级            55
四川省布拖县              45
安徽省亳州市中级            35
安阳县                 34
云南省文山壮族苗族自治州中级      33
福建省尤溪县              31
Name: 识别法院地, dtype: int64


In [ ]:
8.##地级市识别
import pandas as pd
import re

def extract_city_corrected(address):
    
    # 直辖市处理
    municipality_pattern = r'(北京|上海|天津|重庆)(市)?'
    municipality_match = re.search(municipality_pattern, address)
    if municipality_match:
        return f"{municipality_match.group(1)}市"
    
    # 自治州/地区处理
    autonomous_prefecture_pattern = r'([^省]+?自治州)'
    prefecture_pattern = r'([^省]+?地区)'
    
    autonomous_prefecture_match = re.search(autonomous_prefecture_pattern, address)
    if autonomous_prefecture_match:
        return autonomous_prefecture_match.group()
    
    prefecture_match = re.search(prefecture_pattern, address)
    if prefecture_match:
        return prefecture_match.group()
    
    #剥离省级前缀（省/自治区/直辖市）
    province_pattern = r'^[\u4e00-\u9fa5]+?(省|自治区|直辖市)'
    simplified_address = re.sub(province_pattern, '', address).strip() 
    
    #提取地级市
    city_pattern = r'([^区县市]+市)' 
    city_match = re.search(city_pattern, simplified_address)
    
    if city_match:
        return city_match.group()
    
    # 特殊情况：直接以“市”结尾的地址
    if re.search(r'[^省]市$', simplified_address): 
        return simplified_address

    return address  

df = pd.read_excel('C:/Users/82427/Desktop/新建 Microsoft Excel 工作表.xlsx')
df['识别地级市'] = df['识别地级市'].apply(extract_city_corrected)
df.to_excel('C:/Users/82427/Desktop/新建 Microsoft Excel 工作表.xlsx', index=False)

print("处理完成！结果已保存至：新建 Microsoft Excel 工作表.xlsx")


处理完成！结果已保存至：新建 Microsoft Excel 工作表.xlsx


In [3]:
9.###省份识别
import pandas as pd
import re

# 完整的省份和自治区列表
province_list = [
    '北京市', '天津市', '上海市', '重庆市', 
    '河北省', '山西省', '辽宁省', '吉林省', '黑龙江省', 
    '江苏省', '浙江省', '安徽省', '福建省', '江西省', 
    '山东省', '河南省', '湖北省', '湖南省', '广东省', 
    '海南省', '四川省', '贵州省', '云南省', '陕西省', 
    '甘肃省', '青海省', '台湾省', 
    '内蒙古自治区', '广西壮族自治区', '西藏自治区', 
    '宁夏回族自治区', '新疆维吾尔自治区',
    '香港特别行政区', '澳门特别行政区'
]

province_pattern = re.compile(rf'({"|".join(province_list)})')

def extract_province(address):
    """从地址字符串中提取省份信息"""
    if not isinstance(address, str):
        return ""
    match = province_pattern.search(address)
    return match.group(1) if match else ""


try:
    df = pd.read_excel("C:/Users/82427/Desktop/省份识别.xlsx")
    if len(df.columns) < 1:
        raise ValueError("Excel文件中没有足够的列")
    address_column = None
    for col in df.columns:
        if '地址' in col or '市' in col:
            address_column = col
            break
    

    if address_column is None:
        address_column = df.columns[0]
        print(f"警告: 未找到明确的地址列，将使用第一列 '{address_column}'")
    

    df['省份'] = df[address_column].apply(extract_province)
    
    output_path = "C:/Users/82427/Desktop/省份识别_已处理.xlsx"
    df.to_excel(output_path, index=False)
    
    print(f"处理完成，结果已保存到: {output_path}")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

警告: 未找到明确的地址列，将使用第一列 '识别法院地'
处理完成，结果已保存到: C:/Users/82427/Desktop/省份识别_已处理.xlsx


In [2]:
###3拐卖数据审查 是否维持原判
import pandas as pd
import re

df = pd.read_csv("C:/Users/82427/Desktop/二审.csv")

pattern = r"(裁定\s*如下：\s*驳回上诉，\s*维持原判|驳回上述|维持原判|撤回上述)"

df['维持原判'] = df['全文'].apply(lambda x: '维持' if re.search(pattern, str(x)) else '不维持')

df.to_excel("C:/Users/82427/Desktop/二审_处理结果11.xlsx", index=False)

In [ ]:
import pandas as pd

def add_case_info(input_path, output_path):
    df = pd.read_csv(input_path)
    
    if '全文' not in df.columns:
        raise ValueError("CSV文件中缺少必要的'全文'列")
    
    df['全文'] = df['全文'].fillna('').astype(str)
    
    def get_case_info(text):
        has_women = '拐卖妇女罪' in text
        has_child = '拐卖儿童罪' in text
        
        if has_women and has_child:
            return '拐卖妇女儿童'
        elif has_women:
            return '拐卖妇女'
        elif has_child:
            return '拐卖儿童'
        return ''
    
    df['案情信息'] = df['全文'].apply(get_case_info)
    
    df.to_csv(output_path, index=False, encoding='utf_8_sig')
    print(f"处理完成，结果已保存至：{output_path}")

add_case_info("C:/Users/82427/Desktop/拐卖最终版本 - 副本.csv", 
             "C:/Users/82427/Desktop/案情描述2.csv")

处理完成，结果已保存至：C:/Users/82427/Desktop/案情描述2.csv


In [3]:
###统计地级市案件数和被告人数
import pandas as pd

file_path = "C:/Users/82427/Desktop/拐卖新版被告人 （包含不宜公开）.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')
df = df.dropna(subset=['地级市', '年份'])
defendant_counts = df.groupby(['地级市', '年份']).size().reset_index(name='被告人数量')
# 计算各地级市各年的拐卖案件数（案号去重）
case_counts = df.groupby(['地级市', '年份'])['案号'].nunique().reset_index(name='案件数')
result = pd.merge(case_counts, defendant_counts, on=['地级市', '年份']).sort_values(['地级市', '年份'])
result.to_excel("C:/Users/82427/Desktop/拐卖新版被告人删减版.xlsx", index=False)

In [7]:
###面板+省级
import pandas as pd
file_path = "C:/Users/82427/Desktop/裁判文书/拐卖新版被告人 （包含不宜公开）.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')
df = df.dropna(subset=['地级市', '年份'])
prov_defendant = (
    df.groupby(['省份', '年份'])
      .size()
      .reset_index(name='被告人数量')
)
prov_case = (
    df.groupby(['省份', '年份'])['案号']
      .nunique()
      .reset_index(name='案件数')
)
province_result = (
    pd.merge(prov_case, prov_defendant, on=['省份', '年份'])
      .sort_values(['省份', '年份'])
)
province_result.to_excel(
    "C:/Users/82427/Desktop/拐卖新版被告人省级统计（未删减）.xlsx",
    index=False
)


In [5]:
###被害人基本信息
import pandas as pd
import re
import chardet

# 中文数字转阿拉伯数字
def chinese_num_to_digit(cn: str) -> int:
    cn2num = {"零": 0, "一": 1, "二": 2, "两": 2, "三": 3, "四": 4,
              "五": 5, "六": 6, "七": 7, "八": 8, "九": 9, "十": 10}
    if cn.isdigit():
        return int(cn)
    if cn == "十":
        return 10
    if len(cn) == 2 and cn[0] == "十":
        return 10 + cn2num.get(cn[1], 0)
    if len(cn) == 2 and cn[1] == "十":
        return cn2num.get(cn[0], 0) * 10
    if len(cn) == 3 and cn[1] == "十":
        return cn2num.get(cn[0], 0) * 10 + cn2num.get(cn[2], 0)
    return cn2num.get(cn, 0)

# 提取时间年份
def extract_years(text):
    years = set()
    digit_years = re.findall(r"(20\d{2})年", text)
    years.update(map(int, digit_years))
    cn_years = re.findall(r"(二[〇○零一二三四五六七八九十]{3})年", text)
    cn_map = {"〇": "0", "○": "0", "零": "0", "一": "1", "二": "2", "三": "3",
              "四": "4", "五": "5", "六": "6", "七": "7", "八": "8", "九": "9"}
    for cn in cn_years:
        digits = ''.join(cn_map.get(c, '') for c in cn)
        if digits.startswith("20"):
            years.add(int(digits))
    return sorted(years)

# 解析被害人数量并结合时间判断
def extract_victim_details_with_time(text):
    if pd.isna(text):
        return pd.Series([0, 0, 0])

    total, children, women = 0, 0, 0
    years = extract_years(text)
    max_year = max(years) if years else None

    # 儿童
    child_matches = re.findall(r"(20\d{2}|二[〇○零一二三四五六七八九十]{3})?年?.{0,6}?(本案)?[^。；，\d]{0,6}(?:拐卖|贩卖|带走|卖给|抱走)[^。；，\d]{0,6}?([一二三四五六七八九十两\d]+)名?(?:男|女)?童", text)
    for year_text, this_case, num_text in child_matches:
        year = extract_years(year_text) if year_text else []
        if this_case or not year or not max_year or int(''.join(map(str, year))) >= max_year - 1:
            children += chinese_num_to_digit(num_text)

    # 妇女
    women_matches = re.findall(r"(20\d{2}|二[〇○零一二三四五六七八九十]{3})?年?.{0,6}?(本案)?[^。；，\d]{0,6}(?:拐卖|贩卖|带走|卖给|送给)[^。；，\d]{0,6}?([一二三四五六七八九十两\d]+)名?(?:妇女|女子|女性)", text)
    for year_text, this_case, num_text in women_matches:
        year = extract_years(year_text) if year_text else []
        if this_case or not year or not max_year or int(''.join(map(str, year))) >= max_year - 1:
            women += chinese_num_to_digit(num_text)

    total = children + women
    return pd.Series([total, children, women])

# 1. 自动识别编码
file_path = "C:/Users/82427/Desktop/拐卖新版.csv"  
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
encoding_used = result['encoding']
print(f"检测到文件编码为：{encoding_used}")

# 2. 读取文件
df = pd.read_csv(file_path, encoding=encoding_used)

# 3. 提取被害人数量
df[["被害人数量_总计", "被害人数量_儿童", "被害人数量_妇女"]] = df["全文"].apply(extract_victim_details_with_time)

# 4. 保存结果
df.to_csv("C:/Users/82427/Desktop/带被害人数量_已识别.csv", index=False, encoding="utf-8-sig")
print("已保存为：带被害人数量_已识别.csv")



检测到文件编码为：UTF-8-SIG
已保存为：带被害人数量_已识别.csv


In [31]:
import re
import pandas as pd

pattern = re.compile(
    r'(?:拐卖|被拐卖|贩卖)(妇女|儿童)(?:.*?(\d+))',
    re.IGNORECASE | re.DOTALL
)

def extract_victims(text):
    children = 0
    women = 0
    matches = pattern.findall(text)
    for target, count in matches:
        if target == '儿童':
            children += int(count) if count.isdigit() else 0
        elif target == '妇女':
            women += int(count) if count.isdigit() else 0
    return children, women


df = pd.read_csv("C:/Users/82427/Desktop/拐卖新版.csv", encoding="utf-8-sig")
subset = df.iloc[159:163].copy()
subset[['被害人数量_儿童', '被害人数量_妇女']] = subset['全文'].apply(lambda x: pd.Series(extract_victims(x)))
print(subset[['案号', '被害人数量_儿童', '被害人数量_妇女']].to_string(index=False))

             案号  被害人数量_儿童  被害人数量_妇女
  （2014）秀刑初字第4号         0      6039
（2013）红刑初字第365号     10065         0
 （2014）故刑初字第11号      2013         0
（2013）仓刑初字第935号     21045         2
